### Preamble

In [1]:
import $ivy.`org.scalatest::scalatest:3.2.16`
import org.scalatest.{Filter => _, _}, flatspec._, matchers._

import $ivy.$                                

import org.scalatest.{Filter => _, _}, flatspec._, matchers._


# Topic 5: Recursive functions

# Problem 1

Implement a function that drops the first _n_ elements of a list. If the number of elements to be dropped is 0, it must return the same list. The implementation must be tail-recursive.

In [ ]:
class TestDrop(
    drop: (List[Boolean], Int) => List[Boolean]
) extends AnyFlatSpec with should.Matchers:
 
    "drop less elements than the list length" should "return the remaining elements" in:
        
        drop(List(true, false), 0) shouldBe 
            List(true, false)
        
        drop(List(true, false, false, true), 2) shouldBe 
            List(false, true)
        
        drop(List(true, false, true, true, false, true), 3) shouldBe 
            List(true, false, true)
    
    "drop a number of elements greater than or equal to its length" should "return the empty list" in:
        drop(List(), 0) shouldBe 
            List()
    
        drop(List(true, false, true, true, false, true), 6) shouldBe 
            List()

        drop(List(), 2) shouldBe 
            List()
        
        drop(List(true, false, true, true, false, true), 8) shouldBe 
            List()

###### Solution

In [ ]:
@annotation.tailrec
def drop[A](list: List[A], n: Int): List[A] = 
    list match
        case _ :: tail if n > 0 => 
            drop(tail, n-1)
        case list => 
            list

###### Your solution

In [ ]:
run(TestDrop(drop))

# Problem 2

Create a function that counts the number of occurrences of a given element in a list.

In [2]:
class TestOcurrences(
    occurrences: (List[String], String) => Int) 
 extends AnyFlatSpec with should.Matchers:
    
    "occurrences" should "work" in:
        occurrences(List("1","1","1"), "1") shouldBe 3
        occurrences(List("1","2","3"), "2") shouldBe 1
        occurrences(List(), "3") shouldBe 0
        occurrences(List("1","2","3"), "5") shouldBe 0

defined class TestOcurrences

#### Part a) Implement the function recursively, without tail-recursion

###### Solution

In [ ]:
def occurrencesR[A](list: List[A], a: A): Int = 
    list match
        case Nil => 0
        case head :: tail => 
            (if (head == a) 1 else 0) + occurrencesR(tail, a)

Here it's a different implementation that performs the equality test through pattern matching:

In [ ]:
def occurrencesR[A](list: List[A], a: A): Int = 
    list match
        case Nil => 0
        case `a` :: tail => 
            1 + occurrencesR(tail, a)
        case _ :: tail => 
            occurrencesR(tail, a)

###### Your solution

In [ ]:
run(TestOcurrences(occurrencesR[String]))

#### Part b) Implement the function with tail-recursion

out                 aux
---                 --- 
???                 List(1,2,3,4,2,2,1) 
???                 List(2,3,4,2,2,1) 
???                 List(3,4,2,2,1) 
???                 List(4,2,2,1) 
???                 List(2,2,1) 
???                 List(2,1) 
???                 List(1) 
???                 List()

out                 aux
---                 --- 
0                   List(1,2,3,4,2,2,1) 
0                   List(2,3,4,2,2,1) 
1                   List(3,4,2,2,1) 
1                   List(4,2,2,1) 
1                   List(2,2,1) 
2                   List(2,1) 
3                   List(1) 
3                   List()

out                 aux
---                 --- 
0                   List(1,2,3,4,2,2,1) 
0+0                 List(2,3,4,2,2,1) 
0+1                 List(3,4,2,2,1) 
1+0                 List(4,2,2,1) 
1+0                 List(2,2,1) 
1+1                 List(2,1) 
2+1                 List(1) 
3+0                 List()

###### Solution

In [ ]:
def occurrencesTR[A](list: List[A], a: A): Int =
    
    @annotation.tailrec
    def occurrencesAux(acc: Int, list: List[A]): Int = 
        list match
            case Nil => acc
            case head :: tail => 
                occurrencesAux(acc + (if (head == a) 1 else 0), tail)
    
    occurrencesAux(0, list)

###### Your solution

In [3]:
def iterativeTemplate_TailRecursion[A, B](l: List[A]): B = 
    
    def step(out: B, aux: List[A]): B = 
        aux match 
            case Nil => out
            case e :: tail => 
                val out1 = (??? : (B , A) => B)(out, l.head)
                val aux1 = aux.tail
                step(out, aux)

    step(??? : B, l)

defined function iterativeTemplate_TailRecursion

In [7]:
def occurrencesTR[A](l: List[A], e: A): Int = 

    def step(out: Int, aux: List[A]): Int = 
        aux match 
            case Nil => out
            case head :: tail => 
                val out1 = if head == e then out + 1 else out
                val aux1 = aux.tail
                step(out1, aux1)

    step(0 : Int, l)

defined function occurrencesTR

In [10]:
def occurrencesTR[A](l: List[A], e: A): Int = 
    l.foldLeft[Int](0 : Int)(
        (out: Int, head: A) =>  
            if head == e then out + 1 else out: Int
    )

defined function occurrencesTR

In [11]:
def occurrencesFoldLeft[A](l: List[A], e: A): Int = 
    l.foldLeft(0)(
        (out: Int, head: A) =>  
            if head == e then out + 1 else out: Int
    )

defined function occurrencesTR

In [12]:
def occurrencesFoldLeft[A](l: List[A], e: A): Int = 
    l.foldLeft(0):
        case (out, `e`) => out + 1
        case (out, _) => out

defined function occurrencesFoldLeft

In [8]:
run(TestOcurrences(occurrencesTR[String]))

cell2$Helper$TestOcurrences:
occurrences
- should work


# Problem 3

Generate a function that takes the first *n* elements of a list. If the length of the list is less than _n_ it must return the input list (i.e. as many elements as there are).

In [ ]:
class TestTake(
    take: (List[Char], Int) => List[Char]
) extends AnyFlatSpec with should.Matchers:
    
    "take" should "work" in:
        take(List(), 0) shouldBe List()
        take(List(), 5) shouldBe List()
        take(List('1','2','3'), 0) shouldBe List()
        take(List('1','2','3'), 2) shouldBe List('1','2')
        take(List('1','2','3'), 3) shouldBe List('1','2','3')
        take(List('1','2','3'), 10) shouldBe List('1','2','3')

The function does not need to be tail-recursive.

###### Solution

In [ ]:
def take[A](list: List[A], n: Int): List[A] = 
    list match
        case head :: tail if n > 0 => 
            head :: take(tail, n-1)
        case _ => 
            List()

###### Your solution

In [ ]:
run(TestTake(take))

# Problem 4

Write a function that partitions a list of integers into a list of even numbers and a list of odd numbers. 

In [7]:
class TestEvenOddPartition(
    candidate: List[Int] => (List[Int], List[Int])
) extends AnyFlatSpec with should.Matchers:
    
    "partitionEvenOdd" should "work" in:
        candidate(List()) shouldBe (List(), List())
        candidate(List(1,3,5)) shouldBe (List(1,3,5), List())
        candidate(List(0,2,4,6)) shouldBe (List(), List(0,2,4,6))
        candidate(1 :: List(2,3,4,5)) shouldBe {
            val tailSol = (List(3,5), List(2,4))
            // (1 :: List(3,5), List(2,4))
            (1 :: tailSol._1, tailSol._2)
        }

defined class TestEvenOddPartition

Recall that given a [2-tuple](https://www.scala-lang.org/api/current/scala/Tuple2.html) in Scala, we can observe its components as follows:

In [8]:
val t2: (Int, String) = (3, "tres")
val n: Int = t2._1
val s: String = t2._2
val (n1: Int, s1: String) = t2

t2: (Int, String) = (3, "tres")
n: Int = 3
s: String = "tres"

The function does not need to be tail-recursive.

###### Solution

In [ ]:
def partitionEvenOdd(list: List[Int]): (List[Int], List[Int]) = 
    list match
        case Nil => 
            (List(), List())
        case head :: tail => 
            val (odds, evens) = partitionEvenOdd(tail)
            if (head % 2 == 0) (odds, head :: evens)
            else (head :: odds, evens)

###### Your solution

In [5]:
def recursionTemplate[A, B](l: List[A]): B = 
    l match 
        case Nil => ??? : B
        case h :: (t: List[A]) =>  
            val tailSol: B = recursionTemplate(t)
            ??? : B

defined function recursionTemplate

In [4]:
def evenOddPartition(l: List[Int]): (List[Int], List[Int]) = 
    l match 
        case Nil => ??? : (List[Int], List[Int])
        case h :: (t: List[Int]) => 
            val tailSol: (List[Int], List[Int]) = evenOddPartition(t)
            ??? : (List[Int], List[Int])

defined function evenOddPartition

In [4]:
def evenOddPartition(l: List[Int]): (List[Int], List[Int]) = 
    l match 
        case Nil => (Nil : List[Int], Nil : List[Int])
        case h :: (t: List[Int]) => 
            val tailSol: (List[Int], List[Int]) = evenOddPartition(t)
            ??? : (List[Int], List[Int])

defined function evenOddPartition

In [9]:
def evenOddPartition(l: List[Int]): (List[Int], List[Int]) = 
    l match 
        case Nil => (Nil : List[Int], Nil : List[Int])
        case h :: (t: List[Int]) => 
            val tailSol: (List[Int], List[Int]) = evenOddPartition(t)
            if h % 2 == 0 then
                (tailSol._1, h :: tailSol._2)
            else 
                (h :: tailSol._1, tailSol._2)

defined function evenOddPartition

In [11]:
def evenOddPartition(l: List[Int]): (List[Int], List[Int]) = 
    l match 
        case Nil => (Nil, Nil)
        case h :: (t: List[Int]) => 
            val (oddTail, evenTail) = evenOddPartition(t)
            if h % 2 == 0 then
                (oddTail, h :: evenTail)
            else 
                (h :: oddTail, evenTail)

defined function evenOddPartition

In [13]:
def evenOddPartition(l: List[Int]): (List[Int], List[Int]) = 
    l match 
        case Nil => (Nil, Nil)
        case h :: (t: List[Int]) => 
            evenOddPartition(t) match 
                case (oddTail, evenTail) => 
                    if h % 2 == 0 then
                        (oddTail, h :: evenTail)
                    else 
                        (h :: oddTail, evenTail)

defined function evenOddPartition

In [14]:
def evenOddPartition(l: List[Int]): (List[Int], List[Int]) = 
    l match 
        case Nil => (Nil, Nil)
        case h :: (t: List[Int]) => 
            evenOddPartition(t) match 
                case (oddTail, evenTail) if h % 2 == 0 => 
                    (oddTail, h :: evenTail)
                case (oddTail, evenTail) => 
                    (h :: oddTail, evenTail)

defined function evenOddPartition

In [15]:
run(TestEvenOddPartition(evenOddPartition))

cell7$Helper$TestEvenOddPartition:
partitionEvenOdd
- should work


# Problem 5

In [ ]:
class TestSplit(
    split: List[Either[String, Boolean]] => (List[String], List[Boolean]))
 extends AnyFlatSpec with should.Matchers:
    "split" should "work" in:
        split(List(Left("a"), Left("b"), Left("a"))) shouldBe 
            (List("a", "b", "a"), List())
        split(List(Right(false), Right(false), Right(true))) shouldBe 
            (List(), List(false, false, true))
        split(List()) shouldBe
            (List(), List())
        split(List(Right(false), Left("a"), Right(true), Left("b"), Left("a"), Right(true))) shouldBe 
            (List("a", "b", "a"), List(false, true, true))

###### Solution

In [ ]:
def split[A, B](l: List[Either[A, B]]): (List[A], List[B]) = 
    l match
        case Nil => 
            (List(), List())
        case head :: tail =>
            val (leftTail, rightTail) = split(tail)
            head match
                case Left(a) => (a :: leftTail, rightTail)
                case Right(b) => (leftTail, b :: rightTail)

###### Your solution

In [ ]:
run(TestSplit(split))

# Problem 6

Write a funtion that receives a list of pairs of integers and returns a new list made from the sum of all pairs.

In [2]:
class TestSum(
    sum: List[(Int, Int)] => List[Int]
) extends AnyFlatSpec with should.Matchers:
    
    "sum" should "work" in:
        sum(Nil) shouldBe Nil
        sum(List((0,0))) shouldBe List(0)
        sum((1,2) :: List((3,4), (5,6))) shouldBe {
            val tailSol = List(7, 11)
            // List(3, 7, 11)
            // 3 :: List(7, 11)
            (1+2) :: List(7, 11)
        }

defined class TestSum

The function does not need to be tail-recursive.

###### Solution

In [ ]:
def sum(list: List[(Int, Int)]): List[Int] = 
    list match
        case Nil => List()
        case (a,b) :: tail => 
            a+b :: sum(tail)

###### Your solution

In [5]:
def tailRecursionTemplate[A, B](l: List[A]): B = 
    
    def step(out: B, aux: List[A]): B = 
        aux match 
            case Nil => out
            case e :: t => 
                step(???(out, e), t)

    step(???, l)

defined function recursionTemplate

In [5]:
def recursionTemplate[A, B](l: List[A]): B = 
    l match 
        case Nil => ??? : B
        case h :: (t: List[A]) =>  
            val tailSol: B = recursionTemplate(t)
            ??? : B

defined function recursionTemplate

In [24]:
// sum: List[(Int, Int)] => List[Int]
// A = (Int, Int)
// B = List[Int]
def sum(l: List[(Int, Int)]): List[Int] = 
    l match 
        case Nil => Nil : List[Int]
        case (h: (Int, Int)) :: (t: List[(Int, Int)]) =>  
            val tailSol: List[Int] = sum(t)
            (h._1 + h._2) :: tailSol : List[Int]

defined function sum

In [2]:
// sum: List[(Int, Int)] => List[Int]
// A = (Int, Int)
// B = List[Int]
def sum(l: List[(Int, Int)]): List[Int] = 
    l.foldRight[List[Int]](Nil)(
        (h: (Int, Int), tailSol: List[Int]) => 
            (h._1 + h._2) :: tailSol
    )

defined function sum

In [26]:
// sum: List[(Int, Int)] => List[Int]
// A = (Int, Int)
// B = List[Int]
def sum(l: List[(Int, Int)]): List[Int] = 
    l match 
        case Nil => Nil 
        case h :: t =>  
            (h._1 + h._2) :: sum(t)

defined function sum

In [27]:
// sum: List[(Int, Int)] => List[Int]
// A = (Int, Int)
// B = List[Int]
def sum(l: List[(Int, Int)]): List[Int] = 
    l match 
        case Nil => Nil 
        case (a, b) :: t =>  
            a+b :: sum(t)

defined function sum

In [17]:
// sum: List[(Int, Int)] => List[Int]
// A = (Int, Int)
// B = List[Int]
def sum(l: List[(Int, Int)]): List[Int] = 
    l match 
        case Nil => ??? : List[Int]
        case h :: (t: List[(Int, Int)]) =>  
            val tailSol: List[Int] = recursionTemplate(t)
            ??? : List[Int]

defined function sum

In [18]:
// sum: List[(Int, Int)] => List[Int]
// A = (Int, Int)
// B = List[Int]
def and(l: List[(Boolean, Boolean)]): List[Boolean] = 
    l match 
        case Nil => ??? : List[Boolean]
        case h :: (t: List[(Boolean, Boolean)]) =>  
            val tailSol: List[Boolean] = and(t)
            ??? : List[Boolean]

defined function and

In [19]:
// sum: List[(Int, Int)] => List[Int]
// A = (Int, Int)
// B = List[Int]
def map2[A, B, C](l: List[(A, B)], f: (A, B) => C): List[C] = 
    l match 
        case Nil => ??? : List[C]
        case h :: (t: List[(A, B)]) =>  
            val tailSol: List[C] = map2(t, f)
            ??? : List[C]

defined function map2

In [ ]:
// sum: List[(Int, Int)] => List[Int]
val sum: List[(Int, Int)] => List[Int] = 
    ??? 

In [ ]:
def sum(l: (List[Int], List[Int])): List[Int] = 
    ??? 

In [ ]:
def sum(l1: List[Int], l2: List[Int]): List[Int] = 
    ??? 

In [25]:
run(TestSum(sum))

cell21$Helper$TestSum:
sum
- should work


# Problem 7

Write a function that receives two lists and returns a single list whose elements are pairs made from the corresponding elements of each list.

In [ ]:
class TestZip(
    zip: (List[Int], List[Char]) => List[(Int, Char)]
) extends AnyFlatSpec with should.Matchers:
    
    "zip" should "work" in:
        zip(List(), List()) shouldBe List()
        zip(List(), List('a','b')) shouldBe List()
        zip(List(1,2,3), List()) shouldBe List()
        zip(List(1,2,3), List('a','b','c')) shouldBe
            List((1,'a'), (2,'b'), (3, 'c'))
        zip(List(1,2), List('a','b','c')) shouldBe
            List((1,'a'), (2,'b'))
        zip(List(1,2,3), List('a','b')) shouldBe
            List((1,'a'), (2,'b'))

The function does not need to be tail-recursive.

###### Solution

In [ ]:
def zip[A, B](list1: List[A], list2: List[B]): List[(A, B)] = 
    (list1, list2) match
        case (head1 :: tail1, head2 :: tail2) => 
            (head1, head2) :: zip(tail1, tail2)
        case _ => List()

###### Your solution

In [ ]:
run(TestZip(zip[Int, Char]))

# Problem 8

Write a function that returns the greatest element of a list of integers.

In [ ]:
class TestGreatest(
    greatest: List[Int] => Option[Int]
) extends AnyFlatSpec with should.Matchers:
    
    "greatest of an empty list" should "return None" in:
        greatest(List()) shouldBe None
    
    "greatest of a non-empty list" should "return the greatest one" in:
        greatest(List(1,2,3)) shouldBe Some(3)
        greatest(List(3,2,1)) shouldBe Some(3)
        greatest(List(1)) shouldBe Some(1)

Use pattern guards when possible. This is an example of pattern guard:

In [ ]:
val maybeInt: Option[Int] = Some(4)
maybeInt match
    case Some(i) if i>3 => true
    // for any other case, no matter if some(_) or None
    case _ => false

#### Part a) Implement the function recursively, without tail-recursion

###### Solution

In [ ]:
def greatest(list: List[Int]): Option[Int] = 
    list match
        case Nil => None
        case head :: tail => 
            greatest(tail) match
                case Some(e) if e > head => Some(e)
                case _ => Some(head)

###### Your solution

In [ ]:
run(TestGreatest(greatest))

#### Part b) Implement the function with tail-recursion

###### Solution

In [ ]:
def greatestTR(list: List[Int]): Option[Int] =
    
    def greatestAux(out: Option[Int], aux: List[Int]): Option[Int] = 
        aux match
            case Nil => out
            case head :: tail => 
                greatestAux(out match
                    case Some(e) if e > head => Some(e)
                    case _ => Some(head), tail)
    
    greatestAux(None, list)

###### Your solution

In [ ]:
run(TestGreatest(greatestTR))

# Problem 9

Create a function that given a list of strings or integers, returns the concatenation of all the string elements. If the list doesn't contain any string, it must return the empty string.

In [ ]:
class TestConcatenate(
    conc: List[Either[String, Int]] => String
) extends AnyFlatSpec with should.Matchers:
    
    "concatenate" should "work" in:
        conc(List()) shouldBe "" 
        conc(List(Right(1), Right(2), Right(3))) shouldBe ""
        conc(List(Left("hello"), Left(", "), Left("world!"))) shouldBe 
            "hello, world!"
        conc(List(Right(1), Left("hello"), Right(2), 
                  Left(", "), Left("world!"), Right(5))) shouldBe 
            "hello, world!"

Recall the definition of the [`Either[A, B]`](https://www.scala-lang.org/api/current/scala/util/Either.html) algebraic data type in the Scala API. 

The function does not need to be tail-recursive.

###### Solution

In [ ]:
def concatenate(list: List[Either[String, Int]]): String =
    list match
        case Nil => 
            ""
        case Left(s1) :: tail => 
            s1 ++ concatenate(tail)
        case Right(_) :: tail => 
            concatenate(tail)

###### Your solution

In [ ]:
run(TestConcatenate(concatenate))

# Binary trees

The following problems deal with functions on binary trees. This data structure can be defined as the following algebraic data type:

In [28]:
object Std: 
    // type List[A] = 1 + A * List[A]
    enum List[A]: 
        case Nil() 
        case ::(head: A, tail: List[A])

defined object Std

In [30]:
// type Tree[A] = 1 + Tree[A] * A * Tree[A]

enum Tree[A]:
    case Empty()
    case Node(izqdo: Tree[A], root: A, dcho: Tree[A])

import Tree.*

defined class Tree
import Tree.*


         1
       /   \
      2     3 
      \   
       4

In [31]:
val t: Tree[Int] = 
    Node(
        Node(Empty(), 2, Node(Empty(), 4, Empty())),
        1, 
        Node(Empty(), 3, Empty()))

t: Tree[Int] = Node(
  izqdo = Node(
    izqdo = Empty(),
    root = 2,
    dcho = Node(izqdo = Empty(), root = 4, dcho = Empty())
  ),
  root = 1,
  dcho = Node(izqdo = Empty(), root = 3, dcho = Empty())
)

In [16]:
// type Tree[A] = 1 + Tree[A] * A * Tree[A]

enum Tree[A]:
    case Empty()
    case Node(left: Tree[A], root: A, right: Tree[A])

object Tree:
    
    def void[A]: Tree[A] = 
        Empty()
    
    def leaf[A](a: A): Node[A] = 
        Node(Empty(), a, Empty())
    
    def right[A](a: A, tree: Tree[A]): Node[A] = 
        Node(Empty(), a, tree)
    
    def left[A](tree: Tree[A], a: A): Node[A] = 
        Node(tree, a, Empty())
    
    def node[A](left: Tree[A], a: A, right: Tree[A]): Node[A] = 
        Node(left, a, right)

defined class Tree
defined object Tree

The companion object defines some smart constructors that will allow us to write test cases more easily.

In [17]:
import Tree._

import Tree._


In [18]:
val t: Tree[Int] = 
    Node(
        right(2, leaf(4)),
        1, 
        leaf(3))

t: Tree[Int] = Node(
  left = Node(
    left = Empty(),
    root = 2,
    right = Node(left = Empty(), root = 4, right = Empty())
  ),
  root = 1,
  right = Node(left = Empty(), root = 3, right = Empty())
)

In [34]:
val t: Tree[Int] = 
    node(
        right(2, leaf(4)),
        1, 
        leaf(3))

t: Tree[Int] = Node(
  left = Node(
    left = Empty(),
    root = 2,
    right = Node(left = Empty(), root = 4, right = Empty())
  ),
  root = 1,
  right = Node(left = Empty(), root = 3, right = Empty())
)

# Problem 10

Create a function that computes the number of nodes a tree.

In [35]:
class TestTreeNumNodes(
    numNodes: Tree[Int] => Int
) extends AnyFlatSpec with should.Matchers:
    
    "concatenate" should "work" in:
        numNodes(void) shouldBe 0
        numNodes(leaf(1)) shouldBe 1
        numNodes(left(leaf(1), 2)) shouldBe 2
        numNodes(node(leaf(1), 2, leaf(3))) shouldBe 3

defined class TestTreeNumNodes

In [37]:
class TestTreeNumNodes(
    numNodes: Tree[Char] => Int
) extends AnyFlatSpec with should.Matchers:
    
    "concatenate" should "work" in:
        numNodes(void) shouldBe 0
        numNodes(leaf('a')) shouldBe 1
        numNodes(left(leaf('b'), 'c')) shouldBe 2
        numNodes(node(leaf('a'), 'n', leaf('m'))) shouldBe 3

defined class TestTreeNumNodes

###### Solution

In [ ]:
def numNodes[A](tree: Tree[A]): Int = 
    tree match
        case Empty() => 0
        case Node(left, root, right) => 
            1 + numNodes(left) + numNodes(right)

###### Your solution

In [41]:
def numNodes[A](t: Tree[A]): Int = 
    t match 
        case Empty() => 0 : Int 
        case Node(left, root, right) => 
            val leftSol: Int = numNodes(left) 
            val rightSol: Int = numNodes(right)
            leftSol + rightSol + 1 : Int 

defined function numNodes

In [43]:
def recursionTemplate[A, B](t: Tree[A]): B = 
    t match 
        case Empty() => ??? : B 
        case Node(left, root, right) => 
            val leftSol: B = recursionTemplate(left) 
            val rightSol: B = recursionTemplate(right)
            ??? : B // ???(root, leftSol, rightSol)

defined function recursionTemplate

In [42]:
run(TestTreeNumNodes(numNodes))

cell37$Helper$TestTreeNumNodes:
concatenate
- should work


# Problem 11

Create a function that calculates the _height_ of a tree, which is defined as the depth of its deepest node (the depth of a node, in turn, is the number of edges from that node to the root of the tree). An empty tree doesn't have any nodes, so it doesn't have height either. 

In [ ]:
class TestTreeHeight(
    height: Node[Int] => Int
) extends AnyFlatSpec with should.Matchers:
    
    "concatenate" should "work" in:
        height(leaf(1)) shouldBe 0
        height(left(leaf(1),2)) shouldBe 1
        height(node(leaf(1), 2, leaf(3))) shouldBe 1
        height(left(left(leaf(3),2),1)) shouldBe 2

Hint: use the Scala [`max`](https://www.scala-lang.org/api/current/scala/Int.html#max(that:Int):Int) method of the `Int` class to calculate the maximum of two numbers.

###### Solution

In [ ]:
def height[A](tree: Node[A]): Int = 
    tree match
        case Node(Empty(), _, Empty()) => 0
        case Node(Empty(), _, right: Node[A]) => 
            1 + height(right)
        case Node(left: Node[A], _, Empty()) => 
            1 + height(left)
        case Node(left: Node[A], _, right: Node[A]) => 
            1 + height(left) max height(right)

###### Your solution

In [ ]:
run(TestTreeHeight(height))

# Problem 12

A degenerate tree is a tree whose nodes only have a child at most. Create a function that determines whether a tree is degenerate or not.

In [ ]:
class TestIsDegenerate(
    isDegenerate: Tree[Int] => Boolean
) extends AnyFlatSpec with should.Matchers:
    
    "isDegenerate" should "work" in:
        isDegenerate(void) shouldBe true
        isDegenerate(leaf(1)) shouldBe true
        isDegenerate(left(leaf(1), 2)) shouldBe true
        isDegenerate(right(2, leaf(1)))  shouldBe true
        isDegenerate(node(leaf(1), 2, leaf(3))) shouldBe false
        isDegenerate(left(left(leaf(3), 2), 1)) shouldBe true
        isDegenerate(left(right(2, left(right(4, leaf(5)), 3)),1)) shouldBe true
        isDegenerate(left(node(leaf(3), 2, leaf(3)), 1)) shouldBe false

###### Solution

In [ ]:
def isDegenerate[A](tree: Tree[A]): Boolean = 
    tree match
        case Empty() => 
            true
        case Node(Empty(), _, Empty()) => 
            true
        case Node(left: Node[A], _, right: Node[A]) => 
            false
        case Node(Empty(), _, right: Node[A]) => 
            isDegenerate(right)
        case Node(left: Node[A], _, Empty()) => 
            isDegenerate(left)

###### Your solution

In [ ]:
run(TestIsDegenerate(isDegenerate))

# Problem 13

Write a function that returns the leaves of a tree. The leafs of any left child must be placed in the list before any leaf from its right sibling. 

In [44]:
class TestLeaves(
    leaves: Tree[Int] => List[Int]
) extends AnyFlatSpec with should.Matchers:
    
    "leaves" should "work" in:
        leaves(void) shouldBe List()
        leaves(leaf(1)) shouldBe List(1)
        leaves(left(leaf(1), 2)) shouldBe List(1)
        leaves(right(2, leaf(1)))  shouldBe List(1)
        leaves(node(leaf(1), 2, leaf(3))) shouldBe List(1,3)
        leaves(left(left(leaf(3), 2), 1)) shouldBe List(3)
        leaves(left(right(2, left(right(4, leaf(5)), 3)),1)) shouldBe List(5)
        leaves(left(node(leaf(3), 2, leaf(3)), 1)) shouldBe List(3,3)
        leaves(node(node(leaf(1),2,leaf(3)),4,node(leaf(5),6,leaf(7)))) shouldBe
            List(1,3,5,7)

defined class TestLeaves

Hint: use the [`++`](https://www.scala-lang.org/api/current/scala/collection/immutable/List.html#++[B%3E:A](suffix:scala.collection.IterableOnce[B]):CC[B]) method of the `List` class to concatenate lists.

###### Solution

In [ ]:
def leaves[A](tree: Tree[A]): List[A] = 
    tree match
        case Empty() => 
            List()
        case Node(Empty(), a, Empty()) => 
            List(a)
        case Node(left, _, right) => 
            leaves(left) ++ leaves(right)

###### Your solution

In [45]:
def leaves[A](t: Tree[A]): List[A] = 
    ??? 

defined function leaves

In [47]:
def leaves[A, B](t: Tree[A]): List[A] = 
    t match 
        case Empty() => List() : List[A] 
        case Node(left, root, right) => 
            val leftSol: List[A] = leaves(left) 
            val rightSol: List[A] = leaves(right)
            leftSol ++ rightSol : List[A] // ???(root, leftSol, rightSol)

defined function leaves

In [50]:
def leaves[A, B](t: Tree[A]): List[A] = 
    t match 
        case Empty() => List() : List[A] 
        case Node(left, root, right) => 
            val leftSol: List[A] = leaves(left) 
            val rightSol: List[A] = leaves(right)
            if left == Empty() && right == Empty() then 
                List(root) 
            else 
                leftSol ++ rightSol

defined function leaves

In [53]:
def leaves[A, B](t: Tree[A]): List[A] = 
    t match 
        case Empty() => List() : List[A] 
        case Node(left, root, right) => 
            if left == Empty() && right == Empty() then 
                List(root) 
            else 
                val leftSol: List[A] = leaves(left) 
                val rightSol: List[A] = leaves(right)
                leftSol ++ rightSol

defined function leaves

In [54]:
def leaves[A, B](t: Tree[A]): List[A] = 
    t match 
        case Empty() => List() : List[A] 
        case Node(left, root, right) => 
            if left == Empty() && right == Empty() then 
                List(root) 
            else 
                leaves(left) ++ leaves(right)

defined function leaves

In [56]:
def leaves[A, B](t: Tree[A]): List[A] = 
    t match 
        case Empty() => List()
        case Node(Empty(), root, Empty()) => List(root) 
        case Node(left, _, right) => leaves(left) ++ leaves(right)

defined function leaves

In [57]:
leaves(right(1, leaf(4)))

res57: List[Int] = List(4)

In [58]:
run(TestLeaves(leaves))

cell44$Helper$TestLeaves:
leaves
- should work


# Problem 14

This problem deals with [_tree traversals_](https://en.wikipedia.org/wiki/Tree_traversal). 

Hint: use the [`++`](https://www.scala-lang.org/api/current/scala/collection/immutable/List.html#++[B%3E:A](suffix:scala.collection.IterableOnce[B]):CC[B]) method of the `List` class to concatenate lists.

#### Part a) Write a function that creates the pre-order of a binary tree.

In [37]:
class TestPreorder(
    preorder: Tree[Int] => List[Int]
) extends AnyFlatSpec with should.Matchers:
    
    "preorder" should "work" in:
        preorder(void) shouldBe List()
        preorder(leaf(1)) shouldBe List(1)
        preorder(left(leaf(1), 2)) shouldBe List(2,1)
        preorder(right(2, leaf(1)))  shouldBe List(2,1)
        preorder(node(leaf(1), 2, leaf(3))) shouldBe List(2,1,3)
        preorder(left(left(leaf(3), 2), 1)) shouldBe List(1,2,3)
        preorder(node(right(2, left(right(4, leaf(5)), 3)),1, leaf(7))) shouldBe {
            val leftSol: List[Int] = List(2,3,4,5)
            val rightSol: List[Int] = List(7)
            // List(1, 2,3,4,5, 7)
            (1 :: List(2,3,4,5)) ++ List(7)
            (1 :: leftSol) ++ rightSol
        }
        preorder(left(node(leaf(3), 2, leaf(4)), 1)) shouldBe 
            List(1,2,3,4)
        preorder(node(node(leaf(1),2,leaf(3)),4,node(leaf(5),6,leaf(7)))) shouldBe
            List(4,2,1,3,6,5,7)

defined class TestPreorder

###### Solution

In [ ]:
def preorder[A](tree: Tree[A]): List[A] = 
    tree match
        case Empty() => 
            List()
        case Node(left, a, right) => 
            a :: (preorder(left) ++ preorder(right))

###### Your solution

In [39]:
val l: [A] => Tree[A] => List[A] = ??? 

scala.NotImplementedError: an implementation is missing

In [40]:
def preorder[A](t: Tree[A]): List[A] = 
    ??? 

defined function preorder

In [42]:
def preorder[A](t: Tree[A]): List[A] = 
    t match 
        case Empty() => List() : List[A] 
        case Node(left, root, right) => 
            val leftSol: List[A] = preorder(left) 
            val rightSol: List[A] = preorder(right)
            ??? : List[A] // ???(root, leftSol, rightSol)

defined function preorder

In [42]:
def preorder[A](t: Tree[A]): List[A] = 
    t match 
        case Empty() => List() : List[A] 
        case Node(left, root, right) => 
            val leftSol: List[A] = preorder(left) 
            val rightSol: List[A] = preorder(right)
            ??? : List[A] // ???(root, leftSol, rightSol)

defined function preorder

In [41]:
def preorder[A](t: Tree[A]): List[A] = 
    t match 
        case Empty() => ??? : List[A] 
        case Node(left, root, right) => 
            val leftSol: List[A] = preorder(left) 
            val rightSol: List[A] = preorder(right)
            ??? : List[A] // ???(root, leftSol, rightSol)

defined function preorder

In [35]:
List(1,2,3) ++ List(4,5) == List(1,2,3,4,5)
0 :: List(1,2,3) == List(0,1,2,3)
List(4,5,6) :+ 7 == List(4,5,6,7)
List(4,5,6).appended(7) == List(4,5,6,7)
List(4,5,6) ++ List(7) == List(4,5,6,7)

res35_0: Boolean = true
res35_1: Boolean = true
res35_2: Boolean = true
res35_3: Boolean = true
res35_4: Boolean = true

In [43]:
def recursionTemplate[A, B](t: Tree[A]): B = 
    t match 
        case Empty() => ??? : B 
        case Node(left, root, right) => 
            val leftSol: B = recursionTemplate(left) 
            val rightSol: B = recursionTemplate(right)
            ??? : B // ???(root, leftSol, rightSol)

defined function recursionTemplate

In [ ]:
run(TestPreorder(preorder))

#### Part b) Write a function that returns the in-order of a binary tree

In [43]:
class TestInorder(
    inorder: Tree[Int] => List[Int]
) extends AnyFlatSpec with should.Matchers:
    
    "in-order" should "work" in:
        inorder(void) shouldBe List()
        inorder(leaf(1)) shouldBe List(1)
        inorder(left(leaf(1), 2)) shouldBe List(1,2)
        inorder(right(2, leaf(1)))  shouldBe List(2,1)
        inorder(node(leaf(1), 2, leaf(3))) shouldBe List(1,2,3)
        inorder(left(left(leaf(3), 2), 1)) shouldBe List(3,2,1)
        inorder(node(right(2, left(right(4, leaf(5)), 3)),1, leaf(7))) shouldBe 
            val leftInOrder = List(2,4,5,3)
            val rightInOrder = List(7)
            val root = 1
            // List(2,4,5,3, 1, 7)
            // List(2,4,5,3) ++ List(1) ++ List(7)
            (leftInOrder :+   1) ++  rightInOrder
            leftInOrder ++ (1 :: rightInOrder)
        inorder(left(node(leaf(3), 2, leaf(4)), 1)) shouldBe 
            List(3,2,4,1)
        inorder(node(node(leaf(1),2,leaf(3)),4,node(leaf(5),6,leaf(7)))) shouldBe
            List(1,2,3,4,5,6,7)

defined class TestInorder

In [44]:
List(1,2,3) ++ List(4)

res44: List[Int] = List(1, 2, 3, 4)

###### Solution

In [ ]:
def inorder[A](tree: Tree[A]): List[A] = 
    tree match
        case Empty() => 
            List()
        case Node(left, a, right) => 
            inorder(left) ++ (a :: inorder(right))

###### Your solution

In [ ]:
run(TestInorder(inorder))

#### Part c) Write a function that returns post-order of a binary tree

In [ ]:
class TestPostorder(
    postorder: Tree[Int] => List[Int]
) extends AnyFlatSpec with should.Matchers:
    
    "postorder" should "work" in:
        postorder(void) shouldBe List()
        postorder(leaf(1)) shouldBe List(1)
        postorder(left(leaf(1), 2)) shouldBe List(1,2)
        postorder(right(2, leaf(1)))  shouldBe List(1,2)
        postorder(node(leaf(1), 2, leaf(3))) shouldBe List(1,3,2)
        postorder(left(left(leaf(3), 2), 1)) shouldBe List(3,2,1)
        postorder(left(right(2, left(right(4, leaf(5)), 3)),1)) shouldBe 
            List(5,4,3,2,1)
        postorder(left(node(leaf(3), 2, leaf(4)), 1)) shouldBe 
            List(3,4,2,1)
        postorder(node(node(leaf(1),2,leaf(3)),4,node(leaf(5),6,leaf(7)))) shouldBe
            List(1,3,2,5,7,6,4)

###### Solution

In [ ]:
def postorder[A](tree: Tree[A]): List[A] = 
    tree match
        case Empty() => 
            List()
        case Node(left, a, right) => 
            (postorder(left) ++ postorder(right)) appended a

###### Your solution

In [ ]:
run(TestPostorder(postorder))

# Problem 15

Given a tree whose nodes are tuples of integers, write a function that returns a tree of the same shape that contains the sum of the numbers for each node.

In [20]:
class TestSum(
    sum: Tree[(Int, Int)] => Tree[Int]
) extends AnyFlatSpec with should.Matchers:
    
    "sum" should "work" in:
        sum(Empty()) shouldBe 
            Empty()
        sum(void) shouldBe 
            void
        sum(leaf((1,1))) shouldBe 
            leaf(2)
        sum(Node(Empty(), (1,1), Empty())) shouldBe 
            leaf(2)
        sum(left(leaf((1,3)), (2,5))) shouldBe 
            left(leaf(4), 7)
        sum(Node(Empty(), (0,2), Node(Empty(), (-1,2), Empty()))) shouldBe { 
            val root = (0,2)
            val leftSol: Tree[Int] = Empty()
            val rightSol: Tree[Int] = Node(Empty(), 1, Empty())
            // Node(Empty(), 2, Node(Empty(), 1, Empty()))
            Node(leftSol, root._1+root._2, rightSol)
        }
        sum(right((0,2), leaf((-1,2)))) shouldBe 
            // right(2, leaf(1))
            right(2, leaf(1))
        sum(left(left(leaf((-3,6)), (2,0)), (-5,6))) shouldBe 
            left(left(leaf(3), 2), 1)

defined class TestSum

###### Solution

In [ ]:
def sum(tree: Tree[(Int, Int)]): Tree[Int] = 
    tree match
        case Empty() => 
            Empty()
        case Node(left, (i1, i2), right) => 
            Node(sum(left), i1 + i2, sum(right))

###### Your solution

In [8]:
val sum: Tree[(Int, Int)] => Tree[Int] = 
    ??? 

scala.NotImplementedError: an implementation is missing

In [9]:
def sum(t: Tree[(Int, Int)]): Tree[Int] = 
    ??? 

defined function sum

In [21]:
def sum(t: Tree[(Int, Int)]): Tree[Int] = 
    t match 
        case Empty() => Empty() : Tree[Int] 
        case Node(left, root, right) => 
            val leftSol: Tree[Int] = sum(left) 
            val rightSol: Tree[Int] = sum(right)
            Node(leftSol, root._1 + root._2, rightSol) : Tree[Int] // ???(root, leftSol, rightSol)

defined function sum

In [25]:
def sum(t: Tree[(Int, Int)]): Tree[Int] = 
    t match 
        case Empty() => Empty()
        case Node(left, (a, b), right) => 
            Node(sum(left), a+b, sum(right))

defined function sum

In [24]:
def sum(t: Tree[(Int, Int)]): Tree[Int] = 
    t match 
        case Empty() => Empty() : Tree[Int] 
        case Node(left, (a: Int, b: Int), right) => 
            val leftSol: Tree[Int] = sum(left) 
            val rightSol: Tree[Int] = sum(right)
            Node(leftSol, a+b, rightSol) : Tree[Int] // ???(root, leftSol, rightSol)

defined function sum

In [27]:
def sum(t: Tree[(Int, Int)]): Tree[Int] = 
    t match 
        case Empty() => Empty() : Tree[Int] 
        case Node(left, root, right) => 
            root match 
                case (a, b) => 
                    val leftSol: Tree[Int] = sum(left) 
                    val rightSol: Tree[Int] = sum(right)
                    Node(leftSol, a+b, rightSol) : Tree[Int] // ???(root, leftSol, rightSol)

defined function sum

In [26]:
def sum(t: Tree[(Int, Int)]): Tree[Int] = 
    t match 
        case Empty() => Empty() : Tree[Int] 
        case Node(left, root, right) => 
            val leftSol: Tree[Int] = sum(left) 
            val rightSol: Tree[Int] = sum(right)
            Node(leftSol, root._1+root._2, rightSol) : Tree[Int] // ???(root, leftSol, rightSol)

defined function sum

In [11]:
def sum(t: Tree[(Int, Int)]): Tree[Int] = 
    t match 
        case Empty() => ??? : Tree[Int] 
        case Node(left, root, right) => 
            val leftSol: Tree[Int] = sum(left) 
            val rightSol: Tree[Int] = sum(right)
            ??? : Tree[Int] // ???(root, leftSol, rightSol)

defined function sum

In [43]:
def recursionTemplate[A, B](t: Tree[A]): B = 
    t match 
        case Empty() => ??? : B 
        case Node(left, root, right) => 
            val leftSol: B = recursionTemplate(left) 
            val rightSol: B = recursionTemplate(right)
            ??? : B // ???(root, leftSol, rightSol)

defined function recursionTemplate

In [22]:
run(TestSum(sum))

cell20$Helper$TestSum:
sum
- should work
